# Chargement des données

In [2]:
# mise en place des chemins des fichiers contenant les datasets
dossier_data = "data/brutes/"
nom_fichier_application_train = "application_train.csv"
nom_fichier_application_test = "application_test.csv"
nom_fichier_bureau = "bureau.csv"
nom_fichier_bureau_balance = "bureau_balance.csv"
nom_fichier_POS_CASH_balance = "POS_CASH_balance.csv"
nom_fichier_credit_card_balance = "credit_card_balance.csv"
nom_fichier_previous_application = "previous_application.csv"
nom_fichier_installments_payments = "installments_payments.csv"

In [3]:
import pandas as pd

# chargement des datasets (BDD complète)
application_train = pd.read_csv(dossier_data + nom_fichier_application_train)
application_test = pd.read_csv(dossier_data + nom_fichier_application_test)
bureau = pd.read_csv(dossier_data + nom_fichier_bureau)
#bureau_balance = pd.read_csv(dossier_data + nom_fichier_bureau_balance)
#POS_CASH_balance = pd.read_csv(dossier_data + nom_fichier_POS_CASH_balance)
#credit_card_balance = pd.read_csv(dossier_data + nom_fichier_credit_card_balance)
previous_application = pd.read_csv(dossier_data + nom_fichier_previous_application)
#installments_payments = pd.read_csv(dossier_data + nom_fichier_installments_payments)

In [29]:
pd.set_option("display.max_columns", None)
previous_application

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.000000,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,Consumer loans,14704.290,267295.5,311400.0,0.0,267295.5,WEDNESDAY,12,Y,1,0.000000,NaN,NaN,XAP,Approved,-544,Cash through the bank,XAP,NaN,Refreshed,Furniture,POS,XNA,Stone,43,Furniture,30.0,low_normal,POS industry with interest,365243.0,-508.0,362.0,-358.0,-351.0,0.0
1670210,2357031,334635,Consumer loans,6622.020,87750.0,64291.5,29250.0,87750.0,TUESDAY,15,Y,1,0.340554,NaN,NaN,XAP,Approved,-1694,Cash through the bank,XAP,Unaccompanied,New,Furniture,POS,XNA,Stone,43,Furniture,12.0,middle,POS industry with interest,365243.0,-1604.0,-1274.0,-1304.0,-1297.0,0.0
1670211,2659632,249544,Consumer loans,11520.855,105237.0,102523.5,10525.5,105237.0,MONDAY,12,Y,1,0.101401,NaN,NaN,XAP,Approved,-1488,Cash through the bank,XAP,"Spouse, partner",Repeater,Consumer Electronics,POS,XNA,Country-wide,1370,Consumer electronics,10.0,low_normal,POS household with interest,365243.0,-1457.0,-1187.0,-1187.0,-1181.0,0.0
1670212,2785582,400317,Cash loans,18821.520,180000.0,191880.0,NaN,180000.0,WEDNESDAY,9,Y,1,NaN,NaN,NaN,XNA,Approved,-1185,Cash through the bank,XAP,Family,Repeater,XNA,Cash,x-sell,AP+ (Cash loan),-1,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-1155.0,-825.0,-825.0,-817.0,1.0


Au vu des données, je ne vois pas comment utiliser les tables listant les mensualités des crédits précédents pour lesquelles peu d'infos sont disponibles (cf. *POS_CASH_balance*, *bureau_balance* et *installments_payments*). 

De ce fait, les tables qui semblent exploitables sont les suivantes :
- *application_{train|test}*
- *bureau*
- *previous_application*

Les autres ne seront plus chargées dans le notebook.

Avec les données des précédents crédits répertoriés par Credit Bureau, on pourrait voir :
- combien de crédits le client a eu => réel intérêt ? Seulement si on réalise un taux de nb de crédits avec retard de paiement sur nb de crédit, à voir si on a assez de données sur la table principale
- si le client a eu des retards de paiement et si oui de quel ordre
- combien de crédits le client a actuellement (donnée la plus importante parmi les 3)

Alors qu'avec les données des demandes précédentes de crédits faits à Home Credit, on pourrait voir :
- combien de crédits le client a eu
- combien de crédits le client a actuellement (donnée la plus importante parmi les 3)
- combien de demandes de crédit ont déjà été refusées au client par Home Credit

# Préparation des données

## Précédents crédits répertoriés par Credit Bureau

In [41]:
bureau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_CURR              int64  
 1   SK_ID_BUREAU            int64  
 2   CREDIT_ACTIVE           object 
 3   CREDIT_CURRENCY         object 
 4   DAYS_CREDIT             int64  
 5   CREDIT_DAY_OVERDUE      int64  
 6   DAYS_CREDIT_ENDDATE     float64
 7   DAYS_ENDDATE_FACT       float64
 8   AMT_CREDIT_MAX_OVERDUE  float64
 9   CNT_CREDIT_PROLONG      int64  
 10  AMT_CREDIT_SUM          float64
 11  AMT_CREDIT_SUM_DEBT     float64
 12  AMT_CREDIT_SUM_LIMIT    float64
 13  AMT_CREDIT_SUM_OVERDUE  float64
 14  CREDIT_TYPE             object 
 15  DAYS_CREDIT_UPDATE      int64  
 16  AMT_ANNUITY             float64
dtypes: float64(8), int64(6), object(3)
memory usage: 222.6+ MB


On regarde quelles sont les modalités correspondantes à la variable relative au statut du crédit pour savoir si on peut dire si un client a fait défaut de paiement. Etant donné que l'on ne sait pas pourquoi un crédit a été vendu (parce qu'il était mauvais ? Ou pour toute autre raison ?), on ne peut pas savoir dans ce dataset si un client a fait défaut de paiement.

In [33]:
bureau.CREDIT_ACTIVE.value_counts()

Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: CREDIT_ACTIVE, dtype: int64

Sur ce jeu de données, nous allons conserver les variables suivantes :
- SK_ID_CURR
- SK_ID_BUREAU
- CREDIT_ACTIVE
- CREDIT_DAY_OVERDUE
- AMT_CREDIT_SUM
- AMT_CREDIT_SUM_OVERDUE

Elles permettent de créer les données citées dans la partie précédente.

In [44]:
variables_a_conserver = [
    "SK_ID_CURR",
    "SK_ID_BUREAU",
    "CREDIT_ACTIVE",
    "CREDIT_DAY_OVERDUE",
    "AMT_CREDIT_SUM",
    "AMT_CREDIT_SUM_OVERDUE"
]
precedents_credits_CB = bureau[variables_a_conserver].copy()

In [45]:
precedents_credits_CB

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_DAY_OVERDUE,AMT_CREDIT_SUM,AMT_CREDIT_SUM_OVERDUE
0,215354,5714462,Closed,0,91323.00,0.0
1,215354,5714463,Active,0,225000.00,0.0
2,215354,5714464,Active,0,464323.50,0.0
3,215354,5714465,Active,0,90000.00,0.0
4,215354,5714466,Active,0,2700000.00,0.0
...,...,...,...,...,...,...
1716423,259355,5057750,Active,0,11250.00,0.0
1716424,100044,5057754,Closed,0,38130.84,0.0
1716425,100044,5057762,Closed,0,15570.00,0.0
1716426,246829,5057770,Closed,0,36000.00,0.0


## Demandes de crédit en cours chez Home Credit

In [34]:
demandes_credit_train.TARGET.value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [16]:
# Conservation des variables d'entrée pour les modèles
variables_a_conserver = [
    "SK_ID_CURR",
    "TARGET",
    "NAME_CONTRACT_TYPE",
    "CODE_GENDER",
    "FLAG_OWN_CAR",
    "FLAG_OWN_REALTY",
    "CNT_CHILDREN",
    "AMT_INCOME_TOTAL",
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "AMT_GOODS_PRICE",
    "NAME_INCOME_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE",
    "DAYS_BIRTH",
    "DAYS_EMPLOYED",
    "OWN_CAR_AGE",
    "OCCUPATION_TYPE",
    "CNT_FAM_MEMBERS",
    "ORGANIZATION_TYPE",
    "OBS_30_CNT_SOCIAL_CIRCLE",
    "DEF_30_CNT_SOCIAL_CIRCLE",
    "OBS_60_CNT_SOCIAL_CIRCLE",
    "DEF_60_CNT_SOCIAL_CIRCLE"
]

demandes_credit_train = application_train[variables_a_conserver].copy()
variables_a_conserver.remove("TARGET")
demandes_credit_test = application_train[variables_a_conserver].copy()

In [18]:
demandes_credit_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OWN_CAR_AGE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,ORGANIZATION_TYPE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Working,Single / not married,House / apartment,-9461,-637,NaN,Laborers,1.0,Business Entity Type 3,2.0,2.0,2.0,2.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,State servant,Married,House / apartment,-16765,-1188,NaN,Core staff,2.0,School,1.0,0.0,1.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Working,Single / not married,House / apartment,-19046,-225,26.0,Laborers,1.0,Government,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Working,Civil marriage,House / apartment,-19005,-3039,NaN,Laborers,2.0,Business Entity Type 3,2.0,0.0,2.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Working,Single / not married,House / apartment,-19932,-3038,NaN,Core staff,1.0,Religion,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,225000.0,Working,Separated,With parents,-9327,-236,NaN,Sales staff,1.0,Services,0.0,0.0,0.0,0.0
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,225000.0,Pensioner,Widow,House / apartment,-20775,365243,NaN,NaN,1.0,XNA,0.0,0.0,0.0,0.0
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,585000.0,Working,Separated,House / apartment,-14966,-7921,NaN,Managers,1.0,School,6.0,0.0,6.0,0.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,319500.0,Commercial associate,Married,House / apartment,-11961,-4786,NaN,Laborers,2.0,Business Entity Type 1,0.0,0.0,0.0,0.0


In [19]:
demandes_credit_train.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_BIRTH,DAYS_EMPLOYED,OWN_CAR_AGE,CNT_FAM_MEMBERS,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,104582.000000,307509.000000,306490.000000,306490.000000,306490.000000,306490.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,-16036.995067,63815.045904,12.061091,2.152665,1.422245,0.143421,1.405292,0.100049
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,4363.988632,141275.766519,11.944812,0.910682,2.400989,0.446698,2.379803,0.362291
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,-25229.000000,-17912.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,-19682.000000,-2760.000000,5.000000,2.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,-15750.000000,-1213.000000,9.000000,2.000000,0.000000,0.000000,0.000000,0.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,-12413.000000,-289.000000,15.000000,3.000000,2.000000,0.000000,2.000000,0.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,-7489.000000,365243.000000,91.000000,20.000000,348.000000,34.000000,344.000000,24.000000
